In [1]:
import pyomo.environ as pyo
import json
from datetime import timedelta
import polars as pl
from polars  import col as c
import os 
import numpy as np
import math
import tqdm
from datetime import timedelta, datetime, timezone
from optimization_model.optimizaztion_pipeline import first_stage_pipeline
from optimization_model.input_data_prepocessing import generate_first_problem_input_data
from data_display.first_stage_optimization_plots import plot_first_stage_summarized
from data_display.input_data_plots import plot_basin_height_volume_table
from utility.pyomo_preprocessing import extract_optimization_results
from utility.polars_operation import linear_interpolation_for_bound, arange_float
from data_federation.input_model import SmallflexInputSchema
from utility.pyomo_preprocessing import generate_datetime_index, generate_clean_timeseries, optimal_segments, generate_segments, process_performance_table
from config import settings
from utility.general_function import pl_to_dict, build_non_existing_dirs
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
COLORS = px.colors.qualitative.Plotly


from optimization_model.first_model_baseline import generate_baseline_model

In [2]:
os.chdir(os.getcwd().replace("/src", ""))
os.environ['GRB_LICENSE_FILE'] = '/home/ltomasini/gruobi_license/gurobi.lic'


In [3]:
output_file_names: dict[str, str] = json.load(open(settings.OUTPUT_FILE_NAMES))

small_flex_input_schema: SmallflexInputSchema = SmallflexInputSchema()\
.duckdb_to_schema(file_path=output_file_names["duckdb_input"])

Read and validate tables from small_flex_input_data.db file: 100%|████████████████████████████████████████████████████| 12/12 [00:01<00:00,  6.89it/s]


In [11]:
folder_name = ".cache/plot"
build_non_existing_dirs(folder_name)
dict_plot = plot_basin_height_volume_table(small_flex_input_schema=small_flex_input_schema)
fig = dict_plot["Aegina upstream basin"]

fig.update_layout(
    title="Greis lac height-volume curve",
    paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)',
    yaxis=dict(
        nticks=5  # Approximate number of ticks on the y-axis
    ),
    font=dict(
        family="Georgia",
        size=18,
        color="Black"
    )
)
fig.write_image(folder_name + "/griessee_height_volume.png")


In [ ]:
market_country = "CH"
market = "DA"
n_segments = 5

year = 2020
max_flow_factor: float = 1
min_flow_factor: float = 0

solver = pyo.SolverFactory('gurobi')
solver.options['MIPGap'] = 2e-3
for with_pumping, add_wind_production in [(False, False), (True, False), (True, True)]:
    turbined_volume = {}
    pumped_volume = {}
    model: pyo.AbstractModel = generate_baseline_model(with_pumping=with_pumping)
    nb_days_list = [7]
    for nb_days in nb_days_list:
        
        data, turbined_table_per_volume = generate_first_problem_input_data(
            small_flex_input_schema=small_flex_input_schema, 
            hydro_power_plant_name= "Aegina hydro",
            n_segments=n_segments, 
            year=year, 
            max_flow_factor=max_flow_factor,
            min_flow_factor=min_flow_factor, 
            first_time_delta=timedelta(days=nb_days),
            add_wind_production=add_wind_production
        )
        model_instance: pyo.Model = model.create_instance({None: data})
        with tqdm.tqdm(total=1, desc="Solving baseline first stage optimization") as pbar:
            _ = solver.solve(model_instance, load_solutions=True, tee=False)
            pbar.update(1)  
    
        plot_first_stage_summarized(
            with_pumping=with_pumping,
            model_instance=model_instance, 
            min_flow_factor=min_flow_factor, 
            max_flow_factor=max_flow_factor, 
            nb_days=nb_days, 
            year=year)
        
        factor: int = nb_days_list[0]//nb_days
        
        turbined_volume[nb_days] = extract_optimization_results(model_instance, "turbined_volume")\
            .with_columns(c("T")//factor).group_by("T").agg(c("turbined_volume").sum()).sort("T")
        if with_pumping:
            pumped_volume[nb_days] = extract_optimization_results(model_instance, "pumped_volume")\
                .with_columns(c("T")//factor).group_by("T").agg(c("pumped_volume").sum()).sort("T")

Solving baseline first stage optimization: 100%|██████████| 1/1 [00:01<00:00,  1.93s/it]

{'Income': 0.9116625330277445, 'turbined_volume': 23.820974099999983, 'discharge_volume': 23.8209741}


Solving baseline first stage optimization: 100%|██████████| 1/1 [00:12<00:00, 12.05s/it]

{'Income': 0.9894584906062979, 'turbined_volume': 35.382095209999996, 'discharge_volume': 23.8209741, 'pumped_volume': 11.561121109999998}


Solving baseline first stage optimization: 100%|██████████| 1/1 [00:10<00:00, 10.31s/it]

{'Income': 1.2174816308084162, 'turbined_volume': 35.382095209999996, 'discharge_volume': 23.8209741, 'pumped_volume': 11.561121109999998}


In [12]:
turbined_table_per_volume

height,volume,flow,electrical_power,alpha
f64,f64,f64,f64,f64
2350.0,1.39706e6,2.3248,7.2683,3.126419
2351.0,1.62089e6,2.32998,7.30702,3.136087
2352.0,1.86192e6,2.33516,7.34574,3.145712
2353.0,2.11941e6,2.34034,7.38446,3.155294
2354.0,2.39384e6,2.34552,7.42318,3.164833
…,…,…,…,…
2382.0,1.581e7,2.4863,8.504,3.420343
2383.0,1.64183e7,2.4912,8.5425,3.42907
2384.0,1.70266e7,2.4961,8.581,3.437763


In [55]:
basin_volume = extract_optimization_results(model_instance, "basin_volume")

def linear_interpolation_using_cols(data: pl.DataFrame, x_col: str, y_col: str) -> pl.DataFrame:
    x = data[x_col].to_numpy()
    y = data[y_col].to_numpy()
    mask = ~np.isnan(y)
    data = data.with_columns(
        pl.Series(y_col, np.interp(x, x[mask], y[mask])))
    return data

basin_volume = basin_volume.join(
    turbined_table_per_volume[["height", "volume"]], left_on="basin_volume", right_on="volume", how="full", coalesce=True).sort("basin_volume")

result = linear_interpolation_using_cols(basin_volume, "basin_volume", "height").drop_nulls("T").sort("T")
col_names = ["turbined_volume", "pumped_volume", "market_price", "max_market_price", "min_market_price", "discharge_volume"]
result = result.with_columns(
    pl.Series(getattr(model_instance, col).extract_values().values()).alias(col) for col in col_names
)

In [67]:
fig = make_subplots(
        rows=3, cols = 1, shared_xaxes=True, vertical_spacing=0.01, x_title="<b>Weeks<b>", 
        row_titles= ["DA price [EUR/MWh]", "Greis lac height [masl]", "Turbined volume [Mm3]"] )
fig.add_trace(
            go.Scatter(
                x=result["T"].to_list(), y=result["market_price"].to_list(), mode='lines',line=dict(color=COLORS[0]),showlegend=False
            ), row=1, col=1
        ) 
fig.add_trace(
        go.Scatter(
            x=result["T"].to_list(), y=result["max_market_price"].to_list(), mode='lines',line=dict(color="red"),showlegend=False
        ), row=1, col=1
    ) 
fig.add_trace(
        go.Scatter(
            x=result["T"].to_list(), y=result["min_market_price"].to_list(), mode='lines',line=dict(color="red"),showlegend=False
        ), row=1, col=1
    ) 
fig.add_trace(
            go.Scatter(
                x=result["T"].to_list(), y=result["height"].to_list(), mode='lines', line=dict(color=COLORS[0]),showlegend=False
            ), row=2, col=1
        ) 
fig.add_trace(
            go.Bar(
                x=result["T"].to_list(), y=(result["turbined_volume"]/1e6).to_list(), showlegend=False, marker=dict(color=COLORS[0]), 
            ), row=3, col=1
        ) 
if with_pumping:
    fig.add_trace(
                go.Bar(
                    x=result["T"].to_list(), y=(-result["pumped_volume"]/1e6).to_list(), showlegend=False, marker=dict(color="red"), 
                ), row=3, col=1
            )   
fig.update_layout(
    barmode='relative',
    margin=dict(t=60, l=65, r= 10, b=60), 
    width=1200,   # Set the width of the figure
    height=800, 
)  


fig.show()

fig.write_image(folder_name + "/fisrt_results.png")

In [56]:
result

T,basin_volume,height,turbined_volume,pumped_volume,market_price,max_market_price,min_market_price,discharge_volume
i64,f64,f64,f64,f64,f64,f64,f64,f64
0,1.76349e7,2385.0,1.4060e6,0.0,38.84375,51.11,20.45,92892.45
1,1.6322e7,2382.841285,1.4060e6,0.0,43.258631,53.04,32.05,83796.3
2,1.5000e7,2380.664917,1.4060e6,0.0,43.471548,59.44,35.35,75826.8
3,1.3669e7,2378.434876,1.4060e6,0.0,45.46756,66.01,35.39,69421.8
4,1.2333e7,2376.131583,1.4060e6,0.0,37.737262,46.35,20.51,67279.8
…,…,…,…,…,…,…,…,…
48,1.82432e7,2386.0,1.2455e6,0.0,57.092619,110.14,37.15,110217.45
49,1.7108e7,2384.133641,1.4060e6,0.0,58.57131,126.7,28.97,98864.85
50,1.5801e7,2381.984744,0.0,0.0,51.726012,70.07,33.92,85181.25


In [50]:
getattr(model_instance, col_names[0]).extract_values().values()

dict_values([1406039.04, 1406039.04, 1406039.04, 1406039.04, 1406039.04, 1406039.04, 1406039.04, 1406039.04, 1406039.04, 1406039.04, 1406039.04, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1101635.4299999997, 1406039.04, 1406039.04, 1406039.04, 1399013.210000001, 1406039.04, 326527.6000000015, 1406039.04, 389445.2100000009, 1381999.5, 0.0, 1406039.04, 142290.9600000009, 1049148.8000000007, 1406039.04, 1406039.04, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 100159.56999999657, 125570.55000000075, 1245523.580000002, 1406039.04, 0.0, 0.0, 0.0])

In [36]:
y_interp

array([2381., 2381., 2385., 2385., 2385., 2385., 2385., 2385., 2385.,
       2385., 2385., 2385., 2385., 2385., 2385., 2385., 2385., 2385.,
       2385., 2385., 2385., 2385., 2381., 2381., 2381., 2381., 2381.,
       2381., 2381., 2381., 2381., 2381., 2381., 2381., 2381., 2381.,
       2381., 2381., 2381., 2381., 2381., 2381., 2381., 2381., 2381.,
       2381., 2381., 2381., 2381., 2381., 2381., 2381., 2381., 2385.,
       2385., 2381., 2385., 2385., 2385., 2385., 2385., 2385., 2385.,
       2385., 2385., 2385., 2385., 2385., 2385., 2385., 2385., 2385.,
       2385., 2385., 2385., 2385., 2385., 2385., 2385., 2381., 2381.,
       2385., 2385., 2385., 2385., 2385., 2385.])

In [27]:
df["column1"].to_numpy()

array([ 1,  3,  4,  7,  8, 10, 19, 27])

In [ ]:

import polars as pl
import numpy as np

# Sample data
data = {
    'column1': [1, 3, 4, 7, 8, 10 , 19, 27],
    'column2': [2, None, 4, None, 5, 7 , None, 15]  # None represents missing values in Polars
}
df = pl.DataFrame(data)

# Convert columns to NumPy arrays for interpolation


# Mask for non-missing values
mask = ~np.isnan(y)

# Interpolate only for missing values in 'column2'
y_interp = np.interp(x, x[mask], y[mask])

# Create a new DataFrame with interpolated values
df = df.with_columns(pl.Series("column2", y_interp))

# Display the DataFrame after interpolation
print(df)
fig = go.Figure()
fig.add_trace(
        go.Scatter(
            x=df["column1"].to_list(), 
            y=df["column2"].to_list(), 
            showlegend=False, line=dict(color=COLORS[0])
        )
    )


shape: (8, 2)
┌─────────┬───────────┐
│ column1 ┆ column2   │
│ ---     ┆ ---       │
│ i64     ┆ f64       │
╞═════════╪═══════════╡
│ 1       ┆ 2.0       │
│ 3       ┆ 3.333333  │
│ 4       ┆ 4.0       │
│ 7       ┆ 4.75      │
│ 8       ┆ 5.0       │
│ 10      ┆ 7.0       │
│ 19      ┆ 11.235294 │
│ 27      ┆ 15.0      │
└─────────┴───────────┘


In [7]:
wind_production = small_flex_input_schema.power_production_measurement.select(
    "avg_active_power", 
    c("timestamp").dt.year().alias("year"),
    c("timestamp").dt.to_string(format="%m-%d %H:%M").alias("date_str"),
).sort("year").pivot(on="year", values="avg_active_power", index="date_str").sort("date_str")\
.with_columns(
    pl.coalesce("2021", "2024").alias("wind_data")
).select(
    (str(year) + "-" + c("date_str")).str.to_datetime(format="%Y-%m-%d %H:%M").alias("timestamp"),
    "wind_data"
)

fig = go.Figure()
fig.add_trace(
        go.Scatter(
            x=wind_production["timestamp"].to_list(), 
            y=wind_production["wind_data"].to_list(), 
            showlegend=False, line=dict(color=COLORS[0])
        )
    )

In [8]:

nb_days_list = list(turbined_volume.keys())

fig = make_subplots(
        rows=2, cols = 1, shared_xaxes=True, vertical_spacing=0.05, x_title="<b>Week<b>", 
        row_titles= [f"{nb_days_list[0]} days", f"{nb_days_list[1]} days"] )

for i, nb_days in enumerate(list(turbined_volume.keys())):
    fig.add_trace(
            go.Bar(
                x=turbined_volume[nb_days]["T"].to_list(), 
                y=(turbined_volume[nb_days]["turbined_volume"]/1e6).to_list(), 
                showlegend=False, marker=dict(color=COLORS[0]), width=1 
            ), row=i+1, col=1
    )

    fig.add_trace(
            go.Bar(
                x=pumped_volume[nb_days]["T"].to_list(), 
                y=(-pumped_volume[nb_days]["pumped_volume"]/1e6).to_list(), 
                showlegend=False, marker=dict(color="red"), width=1 
            ), row=i+1, col=1
    )


fig.update_layout(
    barmode='relative',
    margin=dict(t=60, l=65, r= 10, b=60), 
    width=1000,   # Set the width of the figure
    height=600,   #
    title= f"Turbined volume [Mm3]")

IndexError: list index out of range

In [ ]:
turbined_volume

{7: shape: (53, 2)
 ┌─────┬─────────────────┐
 │ T   ┆ turbined_volume │
 │ --- ┆ ---             │
 │ i64 ┆ f64             │
 ╞═════╪═════════════════╡
 │ 0   ┆ 1.4060e6        │
 │ 1   ┆ 1.4060e6        │
 │ 2   ┆ 1.4060e6        │
 │ 3   ┆ 1.4060e6        │
 │ 4   ┆ 1.4060e6        │
 │ …   ┆ …               │
 │ 48  ┆ 1.2455e6        │
 │ 49  ┆ 1.4060e6        │
 │ 50  ┆ 0.0             │
 │ 51  ┆ 0.0             │
 │ 52  ┆ 0.0             │
 └─────┴─────────────────┘,
 1: shape: (53, 2)
 ┌─────┬─────────────────┐
 │ T   ┆ turbined_volume │
 │ --- ┆ ---             │
 │ i64 ┆ f64             │
 ╞═════╪═════════════════╡
 │ 0   ┆ 1.4060e6        │
 │ 1   ┆ 1.4060e6        │
 │ 2   ┆ 1.4060e6        │
 │ 3   ┆ 1.4060e6        │
 │ 4   ┆ 1.4060e6        │
 │ …   ┆ …               │
 │ 48  ┆ 602588.16       │
 │ 49  ┆ 401725.44       │
 │ 50  ┆ 451642.86       │
 │ 51  ┆ 0.0             │
 │ 52  ┆ 0.0             │
 └─────┴─────────────────┘}